In [1]:
print("Hello World")

Hello World


In [3]:
from crewai import Crew, Task, Agent
from dotenv import load_dotenv
import os
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'



In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

directory_read_tool = DirectoryReadTool(
    directory="./instructions",
)
file_read_tool = FileReadTool()
search_tool = SerperDevTool()



In [9]:
from crewai_tools import BaseTool
from textblob import TextBlob

class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = "A tool that analyzes the sentiment of a given text to ensure positive and engaging communication"

    def _run(self, text: str) -> str:
        
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return "Positive"
        elif analysis.sentiment.polarity == 0:
            return "Neutral"
        else:
            return "Negative"


sentiment_analysis_tool = SentimentAnalysisTool()

In [10]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [11]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [13]:
crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=True,
    memory=True
)





In [14]:
inputs = {
    "lead_name": "Google",
    "industry": "Technology",
    "key_decision_maker": "Sundar Pichai",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)
print(result)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of Google, a company in the Technology sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: To start compiling a detailed profile on Google, I need to gather information on their company background, key decision-makers, recent business developments, and potential needs that align with our offerings. I will begin by searching the internet to get a broad yet detailed understanding of Google, focusing initially on general company information.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Google company profile detailed informatio

In [16]:
result

CrewOutput(raw="--- Email Draft 1 ---\nSubject: Enhancing Google's AI Prowess with CrewAI's Tailored Solutions\n\nDear Mr. Pichai,\n\nI hope this message finds you well. I am reaching out to congratulate you and the Google team on the successful launch of the advanced AI integrations showcased at Google I/O 2023. Your commitment to transforming technology not only meets current demands but also anticipates future needs in the tech landscape.\n\nAt CrewAI, we are inspired by Google's strides in AI and are excited about the potential synergies between our AI-driven analytics solutions and Google's vision for the future. Our tools are designed to enhance data efficiency and user engagement that can further complement Google's recent AI innovations, particularly in areas like improved spam detection and AI-powered features in Google Maps and Photos.\n\nWe would be thrilled to discuss how CrewAI can partner with Google to drive forward our shared goal of technology excellence through more p

In [23]:
markdown_result = result.raw
print(markdown_result)


--- Email Draft 1 ---
Subject: Enhancing Google's AI Prowess with CrewAI's Tailored Solutions

Dear Mr. Pichai,

I hope this message finds you well. I am reaching out to congratulate you and the Google team on the successful launch of the advanced AI integrations showcased at Google I/O 2023. Your commitment to transforming technology not only meets current demands but also anticipates future needs in the tech landscape.

At CrewAI, we are inspired by Google's strides in AI and are excited about the potential synergies between our AI-driven analytics solutions and Google's vision for the future. Our tools are designed to enhance data efficiency and user engagement that can further complement Google's recent AI innovations, particularly in areas like improved spam detection and AI-powered features in Google Maps and Photos.

We would be thrilled to discuss how CrewAI can partner with Google to drive forward our shared goal of technology excellence through more personalized and immersive